In [1]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import gzip
import os
import json
import gzip

In [2]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [4]:
hotels = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/hotels.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/ratings.csv')
df = hotels.merge(ratings, how="left", on="HotelId")
df.head()

,HotelId,Name,UserId,Review,Rating
0,1,amalia,1,"Amazing hotel, great location and helpful staf...",5
1,1,amalia,2,"For a hotel in a world capital, our room was d...",4
2,1,amalia,3,We are here for 6 nights and found this Hotel ...,5
3,1,amalia,4,"Clean facilities, rooms. Great customer servic...",5
4,1,amalia,5,I have stayed In Athens in several hotels and...,5


In [5]:
del df['Rating']
del df['HotelId']
del df['UserId']
df

,Name,Review
0,amalia,"Amazing hotel, great location and helpful staf..."
1,amalia,"For a hotel in a world capital, our room was d..."
2,amalia,We are here for 6 nights and found this Hotel ...
3,amalia,"Clean facilities, rooms. Great customer servic..."
4,amalia,I have stayed In Athens in several hotels and...
...,...,...
23766,wyndham,I was one of the first guests and everything w...
23767,wyndham,We stayed with our 2 small children on Decembe...
23768,wyndham,Excellent service and view of Athens is the gr...
23769,wyndham,I was in Greece for 1 night. I chose Wyndham G...


In [6]:
df = df.groupby(['Name'])['Review'].apply(','.join).reset_index()
df

,Name,Review
0,InterContinental,"Location not quite in city center, but still o..."
1,amalia,"Amazing hotel, great location and helpful staf..."
2,athens_gate,Helpful and welcoming staff. Clean and conven...
3,attalos,We randomly selected this hotel for our group ...
4,ava,The people at the front desk were amazing. The...
5,central,Clean and comfortable hotel with an excellent ...
6,cypria,very good hotel Big and clean rooms Great loca...
7,electra_metropolis,"Outstanding service, views and spa Everything ..."
8,grand_hyatt,Everything was perfect in this well located ho...
9,hera,This small hotel was ideally located just a ha...


In [7]:
# Κατασκευή ενός pipeline για την εξαγωγή βαθμολογιών λέξεων TF-IDF από το δεδομένο κείμενο χωρίς λέξεις τερματισμού (stopwords)
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,max_features=1000, stop_words= ENGLISH_STOP_WORDS)),('model',KMeans(n_clusters=20))])

In [8]:
# Κάνουμε fit το μοντέλο και το εφαρμόζουμε στα reviews, με αποτέλεσμα να τα κατατάσει σε clusters.
model = pipeline.fit(df['Review'])
a = model.predict(df['Review'])
df['Prediction'] = a 

In [9]:
df.iloc[:,[0,2]]

,Name,Prediction
0,InterContinental,1
1,amalia,13
2,athens_gate,10
3,attalos,19
4,ava,4
5,central,3
6,cypria,9
7,electra_metropolis,14
8,grand_hyatt,6
9,hera,0


In [10]:
# φτιάχνουμε ένα lookup table όπου αποθηκεύουμε κάθε εστιατόριο με το Id του, το ονομάοζουμε lkup_tbl
lkup_tbl = df.iloc[:,[0,2]]
lkup_dict = dict(zip(lkup_tbl.Name, lkup_tbl.Prediction))
lkup_dict

{'InterContinental': 1,
 'amalia': 13,
 'athens_gate': 10,
 'attalos': 19,
 'ava': 4,
 'central': 3,
 'cypria': 9,
 'electra_metropolis': 14,
 'grand_hyatt': 6,
 'hera': 0,
 'herodion': 16,
 'innathens': 2,
 'marble': 5,
 'melia': 8,
 'new': 12,
 'plaka_hotel': 3,
 'polis grand': 18,
 'president': 17,
 'royal': 11,
 'titania': 7,
 'wyndham': 15}

In [11]:
# θέτουμε ένα text, το οποίο θεωρείται ως input
text = ["warm rooms"]

# προβλέπουμε σε πιο cluster ανοίκει το input text με την χρήση του μοντέλου
model = pipeline.predict(text)
model

array([7], dtype=int32)

In [12]:
# απαραίτητες βιβλιοθήκες για να αποθηκεύσουμε το look up table και το μοντέλο
from joblib import load
from joblib import dump

In [18]:
# αποθήκευση του pipeline
dump(pipeline, filename="/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/text_classification1.joblib")

['/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/text_classification1.joblib']

In [19]:
# κατάλληλες βιβλιοθήκες για την φόρτωση του μοντέλου
import pickle
from joblib import load

In [20]:
example_dict = lkup_dict

pickle_out = open("/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/dict.pickle1","wb")
pickle.dump(example_dict, pickle_out)
pickle_out.close()

In [21]:
# συνάρτηση που κάνει recommend
def recommend(text):
  pipeline = load("/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/text_classification1.joblib")
  pickle_in = open("/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/dict.pickle1","rb")
  example_dict = pickle.load(pickle_in)
  out_pred = pipeline.predict(text)
  list_recomm = [id for id, pred in example_dict.items() if pred == out_pred][:20]
  return list_recomm

In [22]:
recommend(["Eggs were amazing!"])

['electra_metropolis']